<a href="https://colab.research.google.com/github/puwadonsri/DADS-7203-NLP/blob/main/Assignment_T5_End_to_End_Question_Generation_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning the 🤗 t5 model on a end-to-end question generation (answer agnostic)
In this notebook, we're going to learn to fine-tune the 🤗 t5 model to **generate questions without providing answers** and use [Weight and Biases](https://wandb.ai/site) for measurements and logs.

### Dataset 🛢️
As dataset we use [SQuAD v1.1](https://rajpurkar.github.io/SQuAD-explorer/): *Stanford Question Answering Dataset is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.*

### Example 🤖
The process:
- You provide the context (the text you want to generate questions from).
- The model generates multiple questions simultaneously.

`Context: 
"Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."`

`Questions:`

- `Who created Python?`,
- `When was Python first released?`
- `What is Python's design philosophy?`

### Requirements
- This is **not an introduction** to Hugging Face Transformer library, it's a **hands-on on how to fine tune t5** for this specific task. 
- If you're not familiar with Hugging Face, **you can watch the HF Course on Transformer models** (it's free) [here](https://huggingface.co/course/chapter1)
- 🏗️ This notebook is a work in progress, some elements (check todo at the end) will change.

### Sources 📚
- [Transformer-based End-to-End Question Generation's Paper](https://arxiv.org/pdf/2005.01107v1.pdf)
- [Patil Suraj's work on question generation](https://github.com/patil-suraj/question_generation/tree/bffa0a51e3ecba3922cafd13f424521135677303)

## Download and install the packages 📦

In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

!pip install tqdm

!pip install wandb

!sudo apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.4 MB/s eta 0:00:0

In [2]:
import torch

from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast

from tqdm import tqdm

from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

import logging
import os
import sys

import numpy as np
import torch

from huggingface_hub import notebook_login

from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)

from google.colab import files

- Connect to Weight and Biases:

In [3]:
import wandb
wandb.login()

%env WANDB_PROJECT=t5-end-to-end-questions-generation

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=t5-end-to-end-questions-generation


## Connect to Hugging Face 🤗
- To be able to share the model in the Hub, we need to **store our authentification token from the Hugging Face website**.


In [4]:
notebook_login()

- Then install Git-lfs and add your mail and username to the config

In [5]:
!git config --global user.email "puwadon.srit@outlook.com"
!git config --global user.name "puwadonsri"

## Loading the dataset 📚
- We use [SQuAD v1.1](https://rajpurkar.github.io/SQuAD-explorer/), but a **modified version** where questions for a context are **concatenated**.
- You need to [download the file here](https://www.simoninithomas.com/hfdataset/squad_modified_for_t5_qg.zip), unzip it and upload it in the next cell.

In [6]:
files.upload()

Saving squad_modified_for_t5_qg.py to squad_modified_for_t5_qg.py


{'squad_modified_for_t5_qg.py': b'# coding=utf-8\r\n# Copyright 2020 The TensorFlow Datasets Authors and the HuggingFace Datasets Authors.\r\n#\r\n# Licensed under the Apache License, Version 2.0 (the "License");\r\n# you may not use this file except in compliance with the License.\r\n# You may obtain a copy of the License at\r\n#\r\n#     http://www.apache.org/licenses/LICENSE-2.0\r\n#\r\n# Unless required by applicable law or agreed to in writing, software\r\n# distributed under the License is distributed on an "AS IS" BASIS,\r\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\r\n# See the License for the specific language governing permissions and\r\n# limitations under the License.\r\n\r\n# Lint as: python3\r\n"""SQUAD: The Stanford Question Answering Dataset."""\r\n"""Modified version for fine tuning T5 on Question Generation """\r\n\r\nimport json\r\n\r\nimport datasets\r\n#from datasets.tasks import QuestionAnsweringExtractive\r\n\r\nlogger = datasets.l

In [7]:
raw_dataset = load_dataset("squad_modified_for_t5_qg.py")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset squad_modified_for_t5_qg downloaded and prepared to /root/.cache/huggingface/datasets/squad_modified_for_t5_qg/plain_text/1.0.0/02ae0815e8483cc76579286179faeb8c8fdbdd328e6741f5c465d9b0bddb8a77. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

- Let see one example of the dataset:

In [8]:
raw_dataset["train"][0]

{'context': 'generate questions: Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'questions': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? {sep_token} What is in front of the Notre Dame Main Building? {sep_token} The Basilica of the Sacred heart at Notre Dame is beside to which structure? {sep_token} What is the Grotto

## Preprocessing the data 🔧
- We first load the model: `"t5-base"` and the `T5TokenizerFast` tokenizer


In [9]:
checkpoint = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


- Because we separate each of our questions with `<sep>` token, we need to add it to the tokenizer tokens.

In [10]:
tokenizer.sep_token = '<sep>'

In [11]:
tokenizer.add_tokens(['<sep>'])
model.resize_token_embeddings(len(tokenizer))

Embedding(32101, 768)

In [12]:
# Check the sep_token_id to verify that it was added to the tokenizer
tokenizer.sep_token_id

32100

- Now, we need to preprocess the data in 3 steps:
1. `add_eos_examples`: Add `</s>` (end of string) at the end of each context and each questions combination.
2. `add_special_tokens`: Replace `{sep_token}` to `<sep>` token between each question.
3. `convert_to_features`: Tokenize the examples with 

In [13]:
max_input_length =  512
max_target_length = 64

In [14]:
# tokenize the examples
def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['context'], 
                                                  max_length=max_input_length, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    target_encodings = tokenizer.batch_encode_plus(example_batch['questions'], 
                                                   max_length=max_target_length, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
  example['context'] = example['context'] + " </s>"
  example['questions'] = example['questions'] + " </s>"
  return example


def add_special_tokens(example):
  example['questions'] = example['questions'].replace("{sep_token}", '<sep>')
  return example

In [15]:
tokenized_dataset  = raw_dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features,  batched=True)

Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset["train"][0]["context"]

'generate questions: Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. </s>'

- Finally, we remove the useless columns `context` and `questions` and we split the tokenized_dataset between train and validation dataset.

In [17]:
tokenized_dataset = tokenized_dataset.remove_columns(
    ["context", "questions"]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [18]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

## Fine-Tuning the t5 model 🧮
- We built a custom DataCollator. A DataCollator **will form a batch using a list of dataset elements as input.** 

In [19]:
# This dataclass implementation is taken from Suraj Patil: https://github.com/patil-suraj/question_generation
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """
    
    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100 
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])
    
    return {
        'input_ids': input_ids, 
        'attention_mask': attention_mask,
        'labels': lm_labels, 
        'decoder_attention_mask': decoder_attention_mask
    }

- We define the `TrainingArguments` object that contains every hyperparameters (learning_rate, nb of epochs...)

In [21]:
training_args = TrainingArguments(output_dir="./gdrive/My Drive/models", 
                                  per_device_train_batch_size=4, 
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=1e-4, 
                                  num_train_epochs=7,
                                  logging_steps=100,
                                  run_name="end2end-questions-generation",
                                  evaluation_strategy="steps",
                                  save_steps=500,
                                  report_to="wandb",
                                  push_to_hub=True,
                                  push_to_hub_model_id="t5-end2end-questions-generation")

In [22]:
logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

# When training is done, we push the fine-tuned model to the Hub
trainer.push_to_hub("t5-end2end-questions-generation")

wandb.finish()

Cloning https://huggingface.co/puwadonsri/t5-end2end-questions-generation into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.5k/850M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.62k/3.62k [00:00<?, ?B/s]

Clean file training_args.bin:  28%|##7       | 1.00k/3.62k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/850M [00:00<?, ?B/s]

...

wandb: Currently logged in as: puwadon-srit (dads7203_nlp_gi5). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,2.588400,1.915886
200,1.970500,1.731005
300,1.843900,1.667226
400,1.742600,1.638182
500,1.714700,1.619873
600,1.690800,1.605257
700,1.631500,1.596661
800,1.627000,1.593896
900,1.612200,1.587745
1000,1.570600,1.586074


Upload file pytorch_model.bin:   0%|          | 1.00/850M [00:00<?, ?B/s]

To https://huggingface.co/puwadonsri/t5-end2end-questions-generation
   04830fe..ba6c294  main -> main

   04830fe..ba6c294  main -> main

To https://huggingface.co/puwadonsri/t5-end2end-questions-generation
   ba6c294..0da7948  main -> main

   ba6c294..0da7948  main -> main



eval/loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▄▇▄▆▅▅▂▁▄▃▆▆▆▄▃▄█▄█▄
eval/samples_per_second,▅▂▅▃▄▄▇█▆▆▃▃▃▄▆▅▁▅▁▅
eval/steps_per_second,▅▂▅▃▄▄▇█▆▆▃▃▃▅▆▆▁▅▁▅
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


## Testing the model 📝
- You can now load the model from HuggingFace and test it.

In [23]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

hfmodel = T5ForConditionalGeneration.from_pretrained("ThomasSimonini/t5-end2end-question-generation")

In [24]:
def hf_run_model(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate questions: " + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  res = hfmodel.generate(input_ids, **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

In [25]:
text = "Harry Potter is a series of seven fantasy novels and eight movies by J. K. Rowling, a British author. It is named for its protagonist and hero, \
Harry Potter. Seven books in the series have sold over 500 million copies across the world in over 70 languages including Latin, \
Welsh and Ancient Greek, and is the best-selling book series of all time. They were adapted into movies by the Warner Bros. The first one was released in 2001. The second, \
third, fourth, and fifth were released respectively in 2002, 2004, 2005 and 2007. The sixth movie, Harry Potter and the Half Blood Prince, was released in 2009. The final \
movie was divided into two parts. The first part, Harry Potter and the Deathly Hallows: Part 1, was released in 2010. Harry Potter and the Deathly Hallows: Part 2 was released in 2011. \
Later, several spin-off movies were released in 2016, 2018, and 2022 respectively, and as well as some TV specials for HBO Max in 2022."

In [26]:
hf_run_model(text)

[['How many books have been sold in the Harry Potter series?',
  ' How many movies have been released in Harry Potter?',
  " What is the name of the protagonist of Harry Potter's books?",
  ' When was the first Harry Potter movie released?',
  ' In what year was Harry Potter and the Deathly Hallows: Part']]

In [27]:
text= "Harry Potter was born in Surrey, England, United Kingdom. His parents, Lily and James Potter, were killed by an evil wizard named Lord Voldemort, \
while Voldemort was trying to kill Harry. Yet Harry escaped with no harm except a scar in the shape of lightning bolt, and Voldemort ends up mostly dead. Harry is raised by his non-magical Aunt Petunia, \
Uncle Vernon, and cousin (Dudley), and is mistreated by them. He believes the false words of the Dursleys that Harry's parents were killed by a car crash. As a child, \
he did not know that he was a wizard. He eventually discovers that there are many magical people, living secretly and hiding from non-wizards (known as Muggles). When Harry turned eleven on July 31, \
he received a letter inviting him to go to a school called Hogwarts for young witches and wizards. Each book tells the story of one year of his life at the school and tells how he struggles. On Hogwarts Express, \
the train they use to get to Hogwarts, he meets Ron Weasley and Hermione Granger, who become his best friends. Voldemort tries several times to return to life and kill Harry. In the fourth book, \
Voldemort comes to life with help from his follower, Peter Pettigrew. The series ends with an epic battle between Harry and Lord Voldemort and a final epilogue that shows Harry and his friends safe and happy with their partners \
and children."

In [28]:
hf_run_model(text)

[['Where was Harry Potter born?',
  " Who killed Harry Potter's parents?",
  ' What did Voldemort try to kill Harry?',
  ' How did Harry escape with no harm except a scar in the shape of a lightning bolt?',
  ' When did Harry turn eleven?',
  '']]

In [29]:
text = "Prayut's nickname is 'Tuu' , and he is known as 'Big Tuu' or 'Uncle Tuu' by his supporters. The opposition has parodied his name, calling him 'Toob'. He is married to Naraporn Chan-o-cha, \
a former associate professor at Chulalongkorn University's Language Institute. She has served as president of the Army Wives' Association since Prayut's selection as army chief in 2010, \
and is involved with distance learning organizations, for whom she teaches English on a long-distance learning television channel. She claimed to have told her husband to cool down when speaking to the media. \
Moreover, she told reporters that she was 'looking after' her husband, taking responsibility for his clothes, makeup and haircut. According to her, Prayut was dressed 'in the English style', wore shoes by Church's \
and suits tailored at 'Broadway'. Prayut and Naraporn have twin daughters, Thanya and Nittha ,twenty-something twin daughters enjoyed brief success a few years ago as a punk-lite pop duo called BADZ"

In [30]:
hf_run_model(text)

[["What is Prayut's nickname?",
  ' What is his nickname by his supporters?',
  ' Who is he married to?',
  ' Where is Naraporn Chan-o-cha a professor?',
  '']]

In [31]:
text = "Prawit Wongsuwan is a Thai politician who is serving as the First Deputy Prime Minister and previously served as Minister of Defense from 2008 to 2011 during the government of Prime Minister Abhisit Vejjajiwa \
and from 2014 to 2019 in the first government of Prime Minister Prayut Chan-ocha. He also previously served as the deputy chairman of the National Council for Peace and Order (NCPO), a military junta which ruled Thailand \
from 2014 to 2019. From 2004 to 2005 he was the commander-in-chief of the Royal Thai Army (RTA). In 2022, he briefly became Acting Prime Minister of Thailand after Prayut was briefly suspended from carrying \
out his duties as Prime Minister of Thailand by the Constitutional Court of Thailand after the court agreed to take up the case on whether Prayut had exceeded the constitutional term limit \
for the office of Prime Minister. The Constitutional Court of Thailand later ruled on 30 September 2022 that Prayut had not exceeded his term limit and may continue to serve as Prime Minister. Prayut was thereafter \
able to resume carrying out duties as Prime Minister as a result of the court ruling and Prawit ceased to become the Acting Prime Minister of Thailand on the same date of the court ruling."

In [32]:
hf_run_model(text)

[['Who is serving as the First Deputy Prime Minister?',
  ' When did Prawit Wongsuwan serve as Minister of Defense?',
  ' Who was the deputy chairman of the National Council for Peace and Order?',
  ' What military junta ruled Thailand from 2014 to 2019?',
  '']]

## What's next?
- **This notebook is a work in progress** , the first next step is to add evaluation test using Rouge metrics, if you don't know about this metric, check this [article](https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460)
- As explained in [the paper](https://arxiv.org/pdf/2005.01107v1.pdf), most of the question are closed questions. This is explained because SQuAD contains 88.26% identification type questions in the training set => **you can improve the model by adding other datasets, by first trying SQuAD v2**
- What about making a webapp? Check [Spaces](https://huggingface.co/spaces)


## My TODO:
- Add Rouge eval test
- Wandb didn't recorded training loss but only evaluation loss.
- Add SQuAD v2
- Pushing the SQuAD version for question generation on HF Hub (instead of using this upload .py file system that's not scalable)
- Solve the issue with Accelerated Inference API => because of the tokenizer

✅ Improve the postprocessing of questions

✅ Make a Spaces web app?
